In [142]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from geopy.geocoders import Nominatim

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer

import Levenshtein as lev
import math
from collections import Counter

from pickle import dump, load
import time
from sklearn.neighbors import BallTree


import itertools
from tqdm.auto import tqdm
tqdm.pandas()
import gc

from fuzzywuzzy import fuzz

In [143]:
start_time = time.time()

In [144]:
pairs = pd.read_pickle('../input/fourpoints-location-matching/train_pairs_raw.pkl')
#pairs = pd.read_csv('../input/foursquare-location-matching/pairs.csv')

In [145]:
#pairs = pairs.sample(frac=0.1, random_state=1, ignore_index=True)
gc.collect()

233

In [146]:
pairs.shape

(57891, 25)

In [147]:
# missing values generate new cols
missing_list = ['url_1','url_2','phone_1','phone_2','address_1','address_2','city_1','city_2','zip_1','zip_2']
for col in tqdm(missing_list):
    pairs[f"{col}_missing"] = pairs[col].notnull().astype('int8')

  0%|          | 0/10 [00:00<?, ?it/s]

In [148]:
def count_occurance(df, cols_1,cols_2):
    for i in tqdm(range(len(cols_1))):
        df[f"{cols_1[i]}_count"] = df[cols_1[i]].map(df[cols_1[i]].dropna().value_counts().to_dict())
        df[f"{cols_2[i]}_count"] = df[cols_2[i]].map(df[cols_2[i]].dropna().value_counts().to_dict())
        df[f"{cols_1[i]}_count_diff"] = (df[f"{cols_2[i]}_count"] - df[f"{cols_1[i]}_count"])
    return df

In [149]:
count_cols_1 = ['country_1','city_1','state_1','categories_1']
count_cols_2 = ['country_2','city_2','state_2','categories_2']
pairs = count_occurance(pairs, count_cols_1, count_cols_2)

  0%|          | 0/4 [00:00<?, ?it/s]

In [150]:
def numeric_group_counts(df,cols_1,cols_2):
    # cols should be [lat,lon]
    for i in tqdm(range(len(cols_1))):
        group_1 = pd.cut(df[cols_1[i]], 180)
        df[f"{cols_1[i]}_count"] = group_1.map(group_1.value_counts().to_dict())
        group_2 = pd.cut(df[cols_1[i]], 180)
        df[f"{cols_2[i]}_count"] = group_2.map(group_2.value_counts().to_dict())
        df[f"{cols_2[i]}_count_diff"] = (df[f"{cols_1[i]}_count"] - df[f"{cols_2[i]}_count"])
    return df

In [151]:
num_group_count_1 = ['latitude_1','longitude_1']
num_group_count_2 = ['latitude_2','longitude_2']
pairs = numeric_group_counts(pairs, num_group_count_1, num_group_count_2)

  0%|          | 0/2 [00:00<?, ?it/s]

In [152]:
# impute missing values
cat_col = pairs.select_dtypes(include = ['object']).columns
pairs[cat_col] = pairs[cat_col].fillna('')

Data preprocessing & Feature transformation:
1. location (latitude, longtitude): finding the distance between two variables
2. word preprocessing: remove url symbol, stop words removal

In [153]:
# 1. location
def distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    d_lon = lon2 - lon1
    d_lat = lat2 - lat1
    a = (np.sin(d_lat/2)) ** 2 + np.cos(lat1) * np.cos(lat2) * (np.sin(d_lon/2)) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance

# lowercase
def lower(df, cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: x.lower())
    return df

# number removing
def num_remove(df, cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: re.sub(r'\d+', '', x))
    return df

# punctuation removal
def punc_remove(df,cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: re.sub('['+string.punctuation+']', ' ', x))
    return df

# white spaces removal
def space_remove(df,cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: x.strip()) # remove front and end space
        df[col] = df[col].str.replace('\s+', ' ', regex=True) # remove double space
    return df

def preprocess(df,cols):
    df[cols] = num_remove(df,cols)[cols]
    df[cols] = punc_remove(df,cols)[cols]
    df[cols] = space_remove(df,cols)[cols]
    return df

# remove url
def remove_URL(df,cols):
    # cols = ["url_1","url_2"]
    df[cols] = df[cols].fillna('')
    for i in tqdm(cols):
        df[i] = df[i].str.replace('http', '')
        df[i] = df[i].str.replace('https', '')
        df[i] = df[i].str.replace('www', '')
        df[i] = df[i].progress_apply(lambda x: re.sub('\W', "", x))
    return df

# stop words removal

def list_to_string(lis):
    string = ''
    for i in tqdm(lis):
        string += i
        string += ' '
    return string[:-1]

def stop(string):
    stops = set(stopwords.words('english'))
    tokens = word_tokenize(string)
    result = [i for i in tqdm(tokens) if not i in stops]
    return result
    
def stop_remove(df,cols):
    stops = set(stopwords.words('english'))
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: ' '.join([word for word in x.split() if word not in stops]))
    return df

In [154]:
lowercase_cols = ['name_1','address_1','city_1','state_1','url_1','categories_1','name_2','address_2','city_2','state_2','url_2','categories_2']
preprocess_cols = ['name_1','address_1','name_2','address_2','url_1','url_2','categories_1','categories_2']
url_columns = ['url_1','url_2']
pairs['distance'] = distance(pairs.latitude_1,pairs.longitude_1,pairs.latitude_2,pairs.longitude_2)

In [155]:
pairs[lowercase_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57891 entries, 0 to 57890
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name_1        57891 non-null  object
 1   address_1     57891 non-null  object
 2   city_1        57891 non-null  object
 3   state_1       57891 non-null  object
 4   url_1         57891 non-null  object
 5   categories_1  57891 non-null  object
 6   name_2        57891 non-null  object
 7   address_2     57891 non-null  object
 8   city_2        57891 non-null  object
 9   state_2       57891 non-null  object
 10  url_2         57891 non-null  object
 11  categories_2  57891 non-null  object
dtypes: object(12)
memory usage: 5.3+ MB


In [156]:
pairs[lowercase_cols] = lower(pairs, lowercase_cols)[lowercase_cols]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

...

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

In [157]:
pairs[preprocess_cols] = preprocess(pairs, preprocess_cols)[preprocess_cols]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

...

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

In [158]:
pairs[url_columns] = remove_URL(pairs, url_columns)[url_columns]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

In [159]:
pairs[preprocess_cols] = stop_remove(pairs, preprocess_cols)[preprocess_cols]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

Count frequency

# text similarity: fuzzy, cosine similarity

In [160]:
def fuzzy_similarity(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    for i in tqdm(range(len(cols_1))):
        df[f"{cols_1[i]}_fuzzy"] = df.progress_apply(lambda x: lev.ratio(x[cols_1[i]],x[cols_2[i]]), axis = 1)
        df[f"{cols_1[i]}_fuzzy_partial"] = df.progress_apply(lambda x: fuzz.partial_ratio(x[cols_1[i]],x[cols_2[i]]), axis = 1)
        df[f"{cols_1[i]}_fuzzy_set"] = df.progress_apply(lambda x: fuzz.token_set_ratio(x[cols_1[i]],x[cols_2[i]]), axis = 1)
    return df    

In [161]:
gc.collect()

135

In [162]:
col_1 = ['name_1','address_1','categories_1','id_1']
col_2 = ['name_2','address_2','categories_2','id_2']
pairs = fuzzy_similarity(pairs, col_1, col_2)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

...

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

  0%|          | 0/57891 [00:00<?, ?it/s]

!pip install -U sentence_transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-roberta-large')

col_1 = ['name_1','address_1','categories_1']
col_2 = ['name_2','address_2','categories_2']
def sen_tran_pca(text_list):
    model = SentenceTransformer('stsb-roberta-large')
    #model = SentenceTransformer('all-MiniLM-L6-v2')
    start_time = time.time()
    #pool = model.start_multi_process_pool()
    #embedding1 = model.encode_multi_process(text_list_1, pool, batch_size =1024)
    embedding = model.encode(text_list, convert_to_tensor=True)
    total_similarity = util.cos_sim(embeddings1, embeddings2)
    #model.stop_multi_process_pool(pool)
    return np.array(total_similarity).diagonal()

def cos_similarity(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    for i in tqdm(range(len(cols_1))):
        df[f"{cols_1[i]}_cos_sim"] = cos_sim(df[cols_1[i]].tolist(),df[cols_2[i]].tolist())
    return df

!pip install -U sentence_transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-roberta-large')
from sklearn.metrics.pairwise import cosine_similarity

col_1 = ['name_1','address_1','categories_1']
col_2 = ['name_2','address_2','categories_2']
def cos_sim(text_list_1,text_list_2):
    model = SentenceTransformer('stsb-roberta-large')
    #model = SentenceTransformer('all-MiniLM-L6-v2')
    start_time = time.time()
    pool = model.start_multi_process_pool()
    embedding1 = model.encode_multi_process(text_list_1, pool, batch_size =1024)
    embedding2 = model.encode(text_list_2, convert_to_tensor=True)
    total_similarity = util.cos_sim(embeddings1, embeddings2)
    model.stop_multi_process_pool(pool)
    return np.array(total_similarity).diagonal()

def cos_similarity(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    for i in tqdm(range(len(cols_1))):
        df[f"{cols_1[i]}_cos_sim"] = cos_sim(df[cols_1[i]].tolist(),df[cols_2[i]].tolist())
    return df

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside','The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome','The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=False)
'''
embeddings2 = model.encode(sentences2, convert_to_tensor=True)
#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))
'''
embeddings1.shape

pairs = cos_similarity(pairs, col_1, col_2)

In [163]:
word_cols = ['name_1','name_2','address_1','address_2','url_1','url_2','categories_1','categories_2']
id_cols = ['id_1','id_2']
pairs[pairs.columns.difference(word_cols)].to_pickle('./train_pairs.pkl')

In [164]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 51.49067544937134 seconds ---


pairs_sample = pairs.iloc[:2,:]
dtype_dict = pairs_sample.dtypes.apply(lambda x: x.name).to_dict()
del pairs_sample
dtype_dict

In [165]:
word_cols = ['name_1','name_2','address_1','address_2','url_1','url_2','categories_1','categories_2']
word_pairs = pairs[word_cols]
word_pairs.to_pickle('./train_pairs_word.pkl')

In [166]:
pairs.head()

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,name_1_fuzzy_set,address_1_fuzzy,address_1_fuzzy_partial,address_1_fuzzy_set,categories_1_fuzzy,categories_1_fuzzy_partial,categories_1_fuzzy_set,id_1_fuzzy,id_1_fuzzy_partial,id_1_fuzzy_set
0,E_b2c967cdef55ff,ja豊橋 石巻支店,34.802585,137.441186,石巻本町東野,豊橋市,愛知県,441-1115,JP,jatoyohashicommapsisetumapaddreisimakiadhtml,...,67,1.000000,100,100,0.300000,40,30,0.3750,38,38
1,E_cbe44b1093e941,domestic terminal,-27.386094,153.120098,bne airport,brisbane airport,qld,4007,AU,bnecomau,...,33,0.000000,0,0,1.000000,100,100,0.2500,25,25
2,E_d90fd1a5d4f408,births deaths general register office,22.278533,114.164479,queensway,admiralty,hk,,HK,,...,0,0.000000,0,0,1.000000,100,100,0.3125,31,31
3,E_63fb661d4e999b,bursa büyükşehir belediye stadyumu,40.210363,29.008948,dikkaldırım mah hayran cad,osmangazi,,16060,TR,sbursaspororgtr,...,24,0.313725,32,33,1.000000,100,100,0.3125,31,31
4,E_c01a9c5cf0f125,camelot,55.744060,37.508564,трц «филион»,moscow,,,RU,,...,0,0.266667,33,29,0.413793,43,34,0.3125,31,31


In [167]:
word_pairs.head()

,name_1,name_2,address_1,address_2,url_1,url_2,categories_1,categories_2
0,ja豊橋 石巻支店,ja豊橋 産直プラザ石巻,石巻本町東野,石巻本町東野,jatoyohashicommapsisetumapaddreisimakiadhtml,jatoyohashicominfoacoopisimakiisimakihtml,banks,farmers markets
1,domestic terminal,runway brissy,bne airport,,bnecomau,,airport terminals,airport terminals
2,births deaths general register office,生死註冊署,queensway,,,,government buildings,government buildings
3,bursa büyükşehir belediye stadyumu,güney tribünü,dikkaldırım mah hayran cad,bursa büyükşehir stadyumu,sbursaspororgtr,bursaspororgtr,soccer stadiums,soccer stadiums
4,camelot,трц «филион»,трц «филион»,багратионовский пр,,filionru,clothing stores,shopping malls


# download data
<a href="train_pairs.csv"> train_pairs </a>

<a href="./train_pairs.pkl"> train_pairs pickle </a>

# Appendix

read large data: https://www.kaggle.com/code/rohanrao/tutorial-on-reading-large-datasets/notebook

read data faster: https://towardsdatascience.com/%EF%B8%8F-load-the-same-csv-file-10x-times-faster-and-with-10x-less-memory-%EF%B8%8F-e93b485086c7